In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((224, 224))
image_size = 0
directory_root = r'C:\Users\User\Desktop\PlantVillage'
width=224
height=224
depth=3

In [3]:
# Função para converter imagens para formato de array

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
# Load the dataset

image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:500]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot ...
[INFO] Processing Corn_(maize)___Common_rust_ ...
[INFO] Processing Corn_(maize)___healthy ...
[INFO] Processing Corn_(maize)___Northern_Leaf_Blight ...
[INFO] Image loading completed


In [5]:
image_size = len(image_list)

In [6]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [7]:
print(label_binarizer.classes_)

['Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot'
 'Corn_(maize)___Common_rust_' 'Corn_(maize)___Northern_Leaf_Blight'
 'Corn_(maize)___healthy']


In [8]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [9]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [10]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [11]:
# Importando a Resnet-50

from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(height,width,depth))

output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)

In [12]:
restnet = Model(restnet.input, outputs=output)

In [13]:
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [14]:
# Criando a fully conected layer

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    width=224
    height=224
    depth=3
    input_shape=(height,width,depth)
    model = Sequential()
    model.add(restnet)
    model.add(Dense(512, input_dim=(224,224,3), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(n_classes))
    model.add(Activation("softmax"))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)

In [16]:
history = model.fit(x_train, y_train, batch_size=BS,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=10, verbose=1
    )

Epoch 1/10
50/50 [==============================] - 264s 5s/step - loss: 2.1654 - accuracy: 0.3216 - val_loss: 0.4496 - val_accuracy: 0.4350
Epoch 2/10
50/50 [==============================] - 252s 5s/step - loss: 0.4633 - accuracy: 0.4777 - val_loss: 0.4074 - val_accuracy: 0.6000
Epoch 3/10
50/50 [==============================] - 272s 5s/step - loss: 0.4284 - accuracy: 0.5116 - val_loss: 0.3939 - val_accuracy: 0.4875
Epoch 4/10
50/50 [==============================] - 254s 5s/step - loss: 0.4199 - accuracy: 0.5321 - val_loss: 0.3828 - val_accuracy: 0.6025
Epoch 5/10
50/50 [==============================] - 252s 5s/step - loss: 0.4055 - accuracy: 0.5626 - val_loss: 0.3757 - val_accuracy: 0.5350
Epoch 6/10
50/50 [==============================] - 270s 5s/step - loss: 0.3902 - accuracy: 0.5666 - val_loss: 0.3765 - val_accuracy: 0.5075
Epoch 7/10
50/50 [==============================] - 254s 5s/step - loss: 0.4007 - accuracy: 0.5803 - val_loss: 0.3126 - val_accuracy: 0.7125
Epoch 8/10
50